In [3]:
import scipy.stats as stats

# Drop missing LossRatio values
anova_data = data.dropna(subset=['LossRatio'])

# Perform ANOVA
groups = [group['LossRatio'].values for name, group in anova_data.groupby('Province')]
f_stat, p_val = stats.f_oneway(*groups)

print(f"ANOVA F-statistic: {f_stat:.4f}, p-value: {p_val:.4f}")

if p_val < 0.05:
    print("Reject H0: Significant risk difference across provinces.")
else:
    print("Fail to reject H0: No significant risk difference across provinces.")


NameError: name 'data' is not defined

In [ ]:
gender_data = data.dropna(subset=['LossRatio', 'Gender'])
male_loss = gender_data[gender_data['Gender']=='Male']['LossRatio']
female_loss = gender_data[gender_data['Gender']=='Female']['LossRatio']

t_stat, p_val = stats.ttest_ind(male_loss, female_loss, equal_var=False)

print(f"T-statistic: {t_stat:.4f}, p-value: {p_val:.4f}")

if p_val < 0.05:
    print("Reject H0: Significant risk difference between genders.")
else:
    print("Fail to reject H0: No significant risk difference between genders.")
